In [1]:
pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 30.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from z3 import Solver, Bool, PbEq, sat

def solve_sudoku(grid):
    """
    Solve a 9×9 Sudoku given as a list of lists of ints.
    Empty cells are 0. Returns a completed grid (new list of lists).
    """
    # Create Bool variables x[i][j][k] meaning "cell (i,j) contains number k+1"
    x = [[[Bool(f"x_{i}_{j}_{k}") for k in range(9)]
           for j in range(9)]
           for i in range(9)]

    s = Solver()

    # F1: Each cell has exactly one number
    for i in range(9):
        for j in range(9):
            # sum_k x[i][j][k] == 1
            s.add(PbEq([(x[i][j][k], 1) for k in range(9)], 1))

    # F2: Each number appears exactly once in each row
    for i in range(9):
        for k in range(9):
            # sum_j x[i][j][k] == 1
            s.add(PbEq([(x[i][j][k], 1) for j in range(9)], 1))

    # F2 (col version): Each number appears exactly once in each column
    for j in range(9):
        for k in range(9):
            # sum_i x[i][j][k] == 1
            s.add(PbEq([(x[i][j][k], 1) for i in range(9)], 1))

    # F3: Each number appears exactly once in each 3×3 block
    for bi in range(3):
        for bj in range(3):
            for k in range(9):
                block_cells = []
                for di in range(3):
                    for dj in range(3):
                        block_cells.append((x[3*bi+di][3*bj+dj][k], 1))
                s.add(PbEq(block_cells, 1))

    # F5: Encode the givens from the input grid
    for i in range(9):
        for j in range(9):
            v = grid[i][j]
            if v != 0:
                # Force x[i][j][v-1] == True
                s.add(x[i][j][v-1])

    # Solve
    if s.check() != sat:
        return "Sudoku is unsatisfiable"
    m = s.model()

    # Extract solution
    solution = [[0]*9 for _ in range(9)]
    for i in range(9):
        for j in range(9):
            for k in range(9):
                if m.evaluate(x[i][j][k]):
                    solution[i][j] = k+1
                    break
    return solution

   

# unsolvable sudoku


In [7]:
puzzle = [
    [0,2,5, 1,0,9, 0,0,0],
    [8,0,2, 0,3,0, 0,6,0],
    [0,3,0, 6,0,0, 7,0,0],

    [5,4,0, 0,1,0, 0,9,0],
    [0,2,0, 0,7,0, 0,0,8],
    [2,0,8, 0,4,0, 0,0,7],

    [0,1,0, 9,0,7, 0,6,0],
    [0,0,9, 0,0,0, 2,8,4],
    [0,0,0, 7,0,6, 0,1,9],
]

sol = solve_sudoku(puzzle)
for row in sol:
    print(" ".join(map(str,row)))


S
u
d
o
k
u
 
i
s
 
u
n
s
a
t
i
s
f
i
a
b
l
e


# solvable sudoku

In [5]:
puzzle = [
    [5,3,0, 0,7,0, 0,0,0],
    [6,0,0, 1,9,5, 0,0,0],
    [0,9,8, 0,0,0, 0,6,0],

    [8,0,0, 0,6,0, 0,0,3],
    [4,0,0, 8,0,3, 0,0,1],
    [7,0,0, 0,2,0, 0,0,6],

    [0,6,0, 0,0,0, 2,8,0],
    [0,0,0, 4,1,9, 0,0,5],
    [0,0,0, 0,8,0, 0,7,9],
]

sol = solve_sudoku(puzzle)
for row in sol:
    print(" ".join(map(str,row)))

5 3 4 6 7 8 9 1 2
6 7 2 1 9 5 3 4 8
1 9 8 3 4 2 5 6 7
8 5 9 7 6 1 4 2 3
4 2 6 8 5 3 7 9 1
7 1 3 9 2 4 8 5 6
9 6 1 5 3 7 2 8 4
2 8 7 4 1 9 6 3 5
3 4 5 2 8 6 1 7 9
